### Glacier Extent Change

ECE471 Remote Sensing Final Project

The Cooper Union

Spring 2021

Ernesto C., Chanoch G., David L.

In [1]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt

### Create a map and add the GLIMS layer on top to visualize glaciers

Link to GLIMS dataset: https://developers.google.com/earth-engine/datasets/catalog/GLIMS_current

In [2]:
Map = geemap.Map()

dataset = ee.FeatureCollection('GLIMS/current')
visParams = {
  'palette': ['gray', 'cyan', 'blue'],
  'min': 0.0,
  'max': 10.0,
  'opacity': 0.8,
}
image = ee.Image().float().paint(dataset, 'area')

# Columbia Glacier, Alaska
lat = 61.1202022
long = -147.1260955

Map.setCenter(long, lat, 9)
Map.addLayer(image, visParams, 'GLIMS/current')
Map.addLayer(dataset, {}, 'for Inspector', False)

# Display map
Map

Map(center=[61.1202022, -147.1260955], controls=(WidgetControl(options=['position'], widget=HBox(children=(Tog…

In [3]:
import json
columbia_glacier_file = 'columbia_glacier.geojson'
with open(columbia_glacier_file, 'r') as f:
  fc = json.load(f)

# grab the WGS geometry
geometry = fc['features'][0]['geometry']

# Create region of interest
roi = ee.Geometry(geometry)

In [4]:
from pprint import pprint
pprint(geometry)

{'coordinates': [[[-147.340617, 60.971976],
                  [-147.340617, 61.174791],
                  [-146.728045, 61.174791],
                  [-146.728045, 60.971976],
                  [-147.340617, 60.971976]]],
 'type': 'Polygon'}


#### Create RGB timelapse

**Note: the add_landsat_ts_gif function will download the gif to your local Downloads folder.
You can change the working directory**

In [5]:
# Create a new map for our timelapse
rgb_timelapse_map = geemap.Map()
rgb_timelapse_map.setCenter(long, lat, 9)
rgb_timelapse_map

Map(center=[61.1202022, -147.1260955], controls=(WidgetControl(options=['position'], widget=HBox(children=(Tog…

In [6]:
# Add timelapse

label = 'Glacier Change'
rgb_timelapse_map.add_landsat_ts_gif(roi=roi,
                                label=label,
                                start_year=2000,
                                end_year=2020,
                                bands=['Red', 'Green', 'Blue'],
                                font_color='red',
                                frames_per_second=0.5,
                                progress_bar_color='blue'
                                )

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\ernes\Downloads\landsat_ts_uia.gif
Adding animated text to GIF ...
Adding GIF to the map ...
The timelapse has been added to the map.


#### Create SWIR, NIR, Green Composite

In [7]:
# Create a new map for our timelapse
timelapse_map = geemap.Map()
timelapse_map.setCenter(long, lat, 10)
timelapse_map

Map(center=[61.1202022, -147.1260955], controls=(WidgetControl(options=['position'], widget=HBox(children=(Tog…

In [8]:
label = 'Glacier Change'
timelapse_map.add_landsat_ts_gif(roi=roi,
                                label=label,
                                start_year=2000,
                                end_year=2020,
                                bands=['SWIR1', 'NIR', 'Green'],
                                font_color='red',
                                frames_per_second=0.5,
                                progress_bar_color='blue'
                                )

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\ernes\Downloads\landsat_ts_bsu.gif
Adding animated text to GIF ...
Adding GIF to the map ...
The timelapse has been added to the map.


### Create an RGB median composite

In [9]:
def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  # Get the pixel QA band.
  qa = image.select('pixel_qa')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

start_date = '2015-05-01'
end_date = '2015-12-31'

dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(start_date, end_date).filterBounds(roi).map(maskL8sr)

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 6000,
  'gamma': 1.4,
}


Map5 = geemap.Map(center=(lat, long), zoom=10)
Map5.addLayer(dataset.median(), visParams)
Map5

Map(center=[61.1202022, -147.1260955], controls=(WidgetControl(options=['position'], widget=HBox(children=(Tog…

### SWIR1, NIR, Green False Color Median Composite

In [10]:
visParams = {
  'bands': ['B6', 'B5', 'B3'],
  'min': 0,
  'max': 6000,
  'gamma': 1.4,
}


Map6 = geemap.Map(center=(lat, long), zoom=10)
Map6.addLayer(dataset.median(), visParams)
Map6

Map(center=[61.1202022, -147.1260955], controls=(WidgetControl(options=['position'], widget=HBox(children=(Tog…